In [142]:
import os
import sys, os
import dill
import sqlalchemy
import ipywidgets as widgets
from aepsych.plotting import plot_strat
from aepsych.server import AEPsychServer
from IPython.display import display
from ipywidgets import HTML, AppLayout, Box, Button, Label, Layout, Text,GridspecLayout
from IPython.core.display import HTML as html

connect_out = widgets.Output(layout=Layout(padding="2px", height="150px"))

database_path = None
with connect_out:
    server = AEPsychServer()
    connect_out.clear_output()
strat = None

# -------- ports ----------
server_ip = widgets.FloatText(
    placeholder="0.0.0.0",
    description='Server IP',
    disabled=False
)
port = widgets.FloatText(
    placeholder="5555",
    description='Port',
    disabled=False
)
connection_inputs = widgets.Box([server_ip, port], 
                               layout=Layout(justify_content="flex-end"))

# ------- View 2 --------
out = widgets.Output(layout=Layout(
    padding="2px", 
    height="100%", 
    align_items='center',
    justify_conten="center",
    width="100%"))
exp_details = widgets.Button(
    description='Exp Details',
    disabled=False,
    button_style='',
    tooltip='Click me')
plot_view = widgets.Button(
    description="Plot View",
    disable=False,
    button_style="",
    tooltip="Click me")
table_view = widgets.Button(
    description="Table View",
    disable=False,
    button_style="",
    tooltip="Click me")
logs = widgets.Button(
    description="Logs",
    disable=False,
    button_style="",
    tooltip="Click me")
interactive = widgets.Button(
    description="Interactive",
    disable=False,
    button_style="",
    tooltip="Click me")
navigation = widgets.Box([exp_details,
plot_view,
table_view,
logs],
layout=widgets.Layout(  
        display='flex',
        flex_flow="column",
        width='200px'))

container_one = widgets.Box([
    Label("Exp Details")],
    layout=widgets.Layout(
    ))
container_two = widgets.Box([
    Label("Plot View")],
    layout=widgets.Layout(
    ))
container_three = widgets.Box([
    Label("Table View")],
    layout=widgets.Layout(
    ))
container_four = widgets.Box([
    Label("Logs")],
    layout=widgets.Layout(
    ))
container_five = widgets.Box([
    Label("Interactive")],
    layout=widgets.Layout(
    ))
nav_container = widgets.Box(
    [navigation,out],
    layout=widgets.Layout(
    display="flex",
    flex_flow='row'
    ))
main_container = widgets.Box(
    [connection_inputs,nav_container],
    layout=widgets.Layout(
    display="flex",
    flex_flow="column"
    )
)
# display(main_container)

def nav_btn_click(b):
    show_display = b.description
    out.clear_output()
    if show_display == 'Exp Details':
        with out: 
            display(container_one)
    elif show_display == "Plot View":
        with out: 
            display(container_two)
    elif show_display == "Table View":
         with out: 
            display(container_three)
    elif show_display == "Logs":
         with out: 
            display(container_four)
    elif show_display == "Interactive":
         with out: 
            display(container_five)

            
exp_details.on_click(nav_btn_click)    
plot_view.on_click(nav_btn_click)
table_view.on_click(nav_btn_click)
logs.on_click(nav_btn_click)
interactive.on_click(nav_btn_click)


#------ View 1 ------            
uploader = widgets.FileUpload(
    description="Connect to Database",accept=".db", multiple=False)

def start_server():
    global strat
    global server
    with error_output:
        try:
            with error_output:
                try: 
                    server = AEPsychServer(database_path=database_path)
                    strat = server.get_strat_from_replay()
                    print("connected")
                except sqlalchemy.exc.DatabaseError as err:
                    connect_out.clear_output()
                    with error_output:
                        print("Unable to connect to server. \n")
                        print(err)
                except RuntimeError as err: 
                    connect_output.clear_output()
                    print("Unable to connect to server. \n")
                    print(err)
#             display(main_container)
        except AssertionError as err: 
            print(err)
            
            

def upload():
    global server
    global strat
    global database_path
    error_output.clear_output() 
    with error_output:     
        error_output.clear_output()
        if uploader.value == {}:
            print("No file uploaded")
        else:
            for file_name, file in uploader.value.items():
                current_path = os.getcwd()
                external_target_path = current_path + "/databases"
                completeName = None 
                
                if os.path.isdir(external_target_path):
                    completeName = os.path.join(external_target_path, file_name)
                else:
                    print("File upload unsuccessful")

                with open(completeName, "wb") as f:
                    f.write(file["content"])
                    if completeName != None:
                        database_path = completeName
                    start_server()
                    f.close()
                    

def uploader_on_change(change): 
    error_output.clear_output()
    with error_output:
        try: 
            upload()
#             start_server()
        except Exception as e:
            exc_type, exc_obj, exc_tb = sys.exc_info()
            fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
            print(exc_type, fname, exc_tb.tb_lineno)

uploader.observe(uploader_on_change, names="value")

error_output = widgets.Output(layout=Layout(
    padding="2px", 
    height="100%", 
    align_items='center',
    justify_conten="center",
    width="100%"))
build_experiment = widgets.Button(
    description='Build Experiment',
    disabled=False,
    button_style='info',
    tooltip='Click me',
    layout=Layout(width='auto', height="auto", margin="15px 40px"))
connect_to_database = widgets.Button(
    description='Connect to Database',
    disabled=False,
    button_style='',
    tooltip='Click me',
    icon="fa-cloud-upload",
    style={"marging": "15px"},
    layout=Layout(width='auto', height='auto', margin="15px 40px"))
home = widgets.Box([
    Label("Build a new experiment, or load data from an old one"), 
    widgets.HBox([build_experiment, uploader], 
                layout=widgets.Layout(
                margin="30px",
                align_items="center"
                ))],
    layout=widgets.Layout(
    display="flex",
    flex_flow='column',
    width="100%",
    height="100%",
    justify_content='center',
    align_items="center",))

display(home, error_output)
# home.layout.display = 'none'

def build():
    error_output.clear_output()
    with error_output: 
        print("Build")
#         home.layout.display = 'none'

    
def connect(): 
    error_output.clear_output()
    with error_output:
        start_server()
#         home.layout.display = 'none'
    
def build_experiment_clicked(b):
    build()

    
def connect_to_database_clicked(b):
    connect()

    
build_experiment.on_click(build_experiment_clicked)
connect_to_database.on_click(connect_to_database_clicked)

Box(children=(Label(value='Build a new experiment, or load data from an old one'), HBox(children=(Button(butto…

Output(layout=Layout(align_items='center', height='100%', padding='2px', width='100%'))